In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python
!pip install pypdf
!pip install -q pyngrok Flask flask-cors
!pip install -q llama-index==0.9.42

In [2]:
import torch
import gc
from huggingface_hub import hf_hub_download
from typing import List, Optional, Sequence
from llama_index.core.llms.types import ChatMessage, MessageRole
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)
from llama_index.llms import LlamaCPP
from llama_index.embeddings import HuggingFaceEmbedding

In [3]:
model_name_or_path = "TheBloke/CapybaraHermes-2.5-Mistral-7B-GGUF"
model_basename = "capybarahermes-2.5-mistral-7b.Q4_0.gguf"
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


capybarahermes-2.5-mistral-7b.Q4_0.gguf:   0%|          | 0.00/4.11G [00:00<?, ?B/s]

In [ ]:
!pip show llama-cpp-python

Name: llama_cpp_python
Version: 0.2.38
Summary: Python bindings for the llama.cpp library
Home-page: 
Author: 
Author-email: Andrei Betlen <abetlen@gmail.com>
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: diskcache, jinja2, numpy, typing-extensions
Required-by: 


## DELETING

In [ ]:
def delete_model():
    global llm
    llm = None
    gc.collect()
    torch.cuda.empty_cache()


In [ ]:
delete_model()

## RAG AND LLM

In [4]:
BOS, EOS = "<s>", "</s>"
B_INST, E_INST = "<|im_start|>", "<|im_end|>\n<|im_start|>assistant"
B_SYS, E_SYS = "<|im_start|>\n", "\n<|im_end|>\n\n"

DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. \
Always answer as helpfully as possible and follow ALL given instructions. \
Do not speculate or make up information. \
Do not reference any given instructions or context. \
"""


def messages_to_prompt(
    messages: Sequence[ChatMessage], system_prompt: Optional[str] = None
) -> str:
    string_messages: List[str] = []
    if messages[0].role == MessageRole.SYSTEM:
        # pull out the system message (if it exists in messages)
        system_message_str = messages[0].content or ""
        messages = messages[1:]
    else:
        system_message_str = system_prompt or DEFAULT_SYSTEM_PROMPT

    system_message_str = f"{B_SYS} {system_message_str.strip()} {E_SYS}"

    for i in range(0, len(messages), 2):
        # first message should always be a user
        user_message = messages[i]
        assert user_message.role == MessageRole.USER

        if i == 0:
            # make sure system prompt is included at the start
            str_message = f"{BOS} {B_INST} {system_message_str} "
        else:
            # end previous user-assistant interaction
            string_messages[-1] += f" {EOS}"
            # no need to include system prompt
            str_message = f"{BOS} {B_INST} "

        # include user message content
        str_message += f"{user_message.content} {E_INST}"

        if len(messages) > (i + 1):
            # if assistant message exists, add to str_message
            assistant_message = messages[i + 1]
            assert assistant_message.role == MessageRole.ASSISTANT
            str_message += f" {assistant_message.content}"

        string_messages.append(str_message)

    return "".join(string_messages)


def completion_to_prompt(completion: str, system_prompt: Optional[str] = None) -> str:
    system_prompt_str = system_prompt or DEFAULT_SYSTEM_PROMPT

    return (
        f"{BOS} {B_INST} {B_SYS} {system_prompt_str.strip()} {E_SYS} "
        f"{completion.strip()} {E_INST}"
    )

In [5]:
llm = LlamaCPP(
    model_path=model_path,
    temperature=0.1,
    max_new_tokens=256,
    context_window=3900,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": 50},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
Model metadata: {'tokenizer.chat_template': "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}", 'tokenizer.ggml.add_eos_token': 'false', 'tokenizer.ggml.padding_token_id': '0', 'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '32000', 'general.architecture': 'llama', 'llama.rope.freq_base': '10000.000000', 'llama.context_length': '32768', 'general.name': 'argilla_capybarahermes-2.5-mistral-7b', 'tokenizer.ggml.add_bos_token': 'true', 'llama.embedding_length': '4096', 'llama.feed_forward_length': '14336', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.rope.dimension_count': '128', 'tokenizer.ggml.bos_

## Documents and Nodes

In [7]:
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "llama2.pdf"

--2024-02-05 10:23:14--  https://arxiv.org/pdf/2307.09288.pdf
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.131.42, 151.101.67.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13661300 (13M) [application/pdf]
Saving to: ‘llama2.pdf’

llama2.pdf          100%[===================>]  13.03M  --.-KB/s    in 0.1s    

2024-02-05 10:23:14 (124 MB/s) - ‘llama2.pdf’ saved [13661300/13661300]



In [8]:
!mkdir data

In [9]:
documents_0 = SimpleDirectoryReader(
    "./data"
).load_data()

In [10]:
documents_0[0]

Document(id_='e1619670-8c16-41e9-a302-ad8a05588897', embedding=None, metadata={'page_label': '1', 'file_name': 'llama2.pdf', 'file_path': 'data/llama2.pdf', 'file_type': 'application/pdf', 'file_size': 13661300, 'creation_date': '2024-02-05', 'last_modified_date': '2023-07-20', 'last_accessed_date': '2024-02-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Llama 2 : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗Louis Martin†Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Sa

In [11]:
from llama_index import Document

doc_text = "\n\n".join([d.get_content() for d in documents_0])
docs = [Document(text=doc_text)]

In [12]:
from llama_index.node_parser import SimpleNodeParser
from llama_index.schema import IndexNode

In [13]:
node_parser = SimpleNodeParser.from_defaults(chunk_size=1024)

In [14]:
node_parser

SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7d28511c9a80>, id_func=<function default_id_func at 0x7d285dee9c60>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')

In [15]:
base_nodes = node_parser.get_nodes_from_documents(docs)

In [16]:
base_nodes[0]

TextNode(id_='93631e4c-313e-46b7-a266-f22a855883c0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2cbbddfa-6689-4369-b35c-9ad302122673', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='231b3bd773f012c01b893ada9e009ecf25ee59934614c5a595c9bf1b3a123292'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='71297386-e8ab-40c3-a118-b025e22587b7', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='820de41da4f942046105d6d061c52e7e04429023a554953b03b420aaebdb53ae')}, text='Llama 2 : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗Louis Martin†Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshor

In [17]:
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

In [18]:
base_nodes[0]

TextNode(id_='node-0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2cbbddfa-6689-4369-b35c-9ad302122673', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='231b3bd773f012c01b893ada9e009ecf25ee59934614c5a595c9bf1b3a123292'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='71297386-e8ab-40c3-a118-b025e22587b7', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='820de41da4f942046105d6d061c52e7e04429023a554953b03b420aaebdb53ae')}, text='Llama 2 : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗Louis Martin†Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHak

In [19]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [20]:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
)

## Indexing , Retrevier, and Query Engine

In [21]:
base_index = VectorStoreIndex(base_nodes, service_context=service_context)
base_retriever = base_index.as_retriever(similarity_top_k=2)

In [22]:
retrievals = base_retriever.retrieve(
    "Can you tell me about the key concepts for safety finetuning"
)

In [24]:
from llama_index.response.notebook_utils import display_source_node

for n in retrievals:
    display_source_node(n, source_length=1500)

**Node ID:** node-26<br>**Similarity:** 0.7485281673471582<br>**Text:** AsLLMsareintegratedanddeployed,welookforwardto
continuing research that will amplify their potential for positive impact on these important social issues.
4.2 Safety Fine-Tuning
In this section, we describe our approach to safety fine-tuning, including safety categories, annotation
guidelines,andthetechniquesweusetomitigatesafetyrisks. Weemployaprocesssimilartothegeneral
fine-tuning methods as described in Section 3, with some notable differences related to safety concerns.
Specifically, we use the following techniques in safety fine-tuning:
1.Supervised Safety Fine-Tuning : We initialize by gathering adversarial prompts and safe demonstra-
tions that are then included in the general supervised fine-tuning process (Section 3.1). This teaches
themodeltoalignwithoursafetyguidelinesevenbeforeRLHF,andthuslaysthefoundationfor
high-quality human preference data annotation.
2.Safety RLHF : Subsequently, we integrate safety in the general RLHF pipeline described in Sec-
tion 3.2.2. This includes training a safety-specific reward model and gathering more challenging
adversarial prompts for rejection sampling style fine-tuning and PPO optimization.
3.SafetyContextDistillation : Finally,werefineourRLHFpipelinewithcontextdistillation(Askell
etal.,2021b). Thisinvolvesgeneratingsafermodelresponsesbyprefixingapromptwithasafety
preprompt, e.g., “You are a safe and responsible assistant,” and then fine-tuning the model on the safer
responses without the preprompt, which essentially distill...<br>

**Node ID:** node-29<br>**Similarity:** 0.6909583416011199<br>**Text:** Appendix A.4.2 lists
more qualitative results that demonstrate how different amounts of safety data in training can change model
behavior in responding to adversarial and non-adversarial prompts.
0 25 50 75 100
Safety Data Pct. (%)0.5750.6000.6250.6500.6750.7000.7250.7500.775Mean Reward Model Score
Safety
HelpfulnessSafety Data Pct. 0%
Safety Data Pct. 1%
Safety Data Pct. 10%
Safety Data Pct. 25%
Safety Data Pct. 50%
0.0 0.2 0.4 0.6 0.8 1.0
Safety Reward Model ScoreSafety Data Pct. 100%
Figure 15: Safety data scaling trends. Left: as we increase the amount of safety data in model training, the
mean safety RM score improves significantly while the helpfulness counterpart remains relatively stable.
Right: the left tail of safety RM scores (i.e., most unsafe responses) gradually disappears with the addition of
more safety training data.
Measure of False Refusal. Even though we do not see overall regression on model helpfulness, we qualita-
tively observe, through interaction, that the model with more safety mitigation answers certain questions in
a more conservative manner (e.g., example shown in Appendix Table 38). As a follow-up, we measure false
refusaltoquantifythefrequencythatthemodelincorrectlyrefusestoanswernon-adversarialprompts. Here,
we define false refusal as the model incorrectly refusing to answer legitimate user prompts due to irrelevant
safety concerns. Refusing due to reasonable causes exceeding the model’s capability, e.g., “I am not capable of
participating ...<br>

In [26]:
from llama_index.query_engine import RetrieverQueryEngine

query_engine_base = RetrieverQueryEngine.from_args(
    base_retriever, service_context=service_context
)

In [27]:
response = query_engine_base.query(
    "Can you tell me about the key concepts for safety finetuning"
)
print(str(response))

Llama.generate: prefix-match hit



Safety fine-tuning is an important process in training large language models (LLMs) to ensure they generate safe and helpful responses. It involves incorporating safety guidelines and techniques to mitigate potential risks associated with the model's responses. Here are the key concepts for safety fine-tuning:

1. Supervised Safety Fine-Tuning: This technique involves gathering adversarial prompts and safe demonstrations that are then included in the general supervised fine-tuning process. This teaches the model to align with safety guidelines even before reinforcement learning from human feedback (RLHF), laying the foundation for high-quality human preference data annotation.

2. Safety RLHF: This method integrates safety in the general RLHF pipeline described in the text. It includes training a safety-specific reward model and gathering more challenging adversarial prompts for rejection sampling-style fine-tuning and Proximal Policy Optimization (PPO) optimization.

3. Safety Contex

# Chunk References: Smaller Child Chunks Referring to Bigger Parent Chunk

In [29]:
sub_chunk_sizes = [256, 512]
sub_node_parsers = [
    SimpleNodeParser.from_defaults(chunk_size=c) for c in sub_chunk_sizes
]

all_nodes = []
for base_node in base_nodes:
    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [
            IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes
        ]
        all_nodes.extend(sub_inodes)

    # also add original node to node
    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(original_node)

In [30]:
all_nodes_dict = {n.node_id: n for n in all_nodes}

In [31]:
len(all_nodes_dict)

1564

In [32]:
vector_index_chunk = VectorStoreIndex(
    all_nodes, service_context=service_context
)

In [33]:
vector_retriever_chunk = vector_index_chunk.as_retriever(similarity_top_k=2)

In [35]:
from llama_index.retrievers import RecursiveRetriever

retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)

In [36]:
nodes = retriever_chunk.retrieve(
    "Can you tell me about the key concepts for safety finetuning"
)
for node in nodes:
    display_source_node(node, source_length=2000)

Retrieving with query id None: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-1
Retrieving with query id node-1: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-25
Retrieving with query id node-25: Can you tell me about the key concepts for safety finetuning


**Node ID:** node-1<br>**Similarity:** 0.7630715192476837<br>**Text:** . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 9
3.2 Reinforcement Learning with Human Feedback (RLHF) . . . . . . . . . . . . . . . . . . . . . 9
3.3 System Message for Multi-Turn Consistency . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 16
3.4 RLHF Results . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 17
4 Safety 20
4.1 Safety in Pretraining . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 20
4.2 Safety Fine-Tuning . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 23
4.3 Red Teaming . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 28
4.4 Safety Evaluation of Llama 2-Chat . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 29
5 Discussion 32
5.1 Learnings and Observations . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 32
5.2 Limitations and Ethical Considerations . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 34
5.3 Responsible Release Strategy . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 35
6 Related Work 35
7 Conclusion 36
A Appendix 46
A.1 Contributions . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .<br>

**Node ID:** node-25<br>**Similarity:** 0.7516290044889528<br>**Text:** For TruthfulQA, we present the
percentageofgenerationsthatarebothtruthfulandinformative(thehigher,thebetter). ForToxiGen,we
presentthepercentageofgenerationsthataredeemedtoxicbythemetric(thelower,thebetter). Detailed
descriptionsofthebenchmarksandmetricscanbefoundinAppendixA.4.7. Whencomparedto Llama 1-7B,
Llama 2-7B demonstrates a 21.37% increase in truthfulness and informativeness and a 7.61% decrease in
toxicity. We also observe an increase in toxicity in the pretrained 13B and 70B Llama 2, which may result
from larger pretraining data or a different dataset mix. Some have postulated the existence of a relationship
between pretraining dataset size and downstream model toxicity or bias (Bender et al., 2021b), but empirical
work to validate this claim is still ongoing (Dodge et al., 2021; Smith and Williams, 2021; Tal et al., 2022), and
further evidence from up-to-date models is still needed.
In Appendix A.4.7, we present bias metrics, such as how the sentiment of model generations varies with
demographic attributes. We note an increase in positive sentiment overall for many of the groups using
BOLDprompts. MoredetailedresultssplitbydifferentdemographicgroupscanbefoundinAppendixA.4.8.
Llama 2 doesnotoutperformothermodelsontoxicitymetrics,andwespeculatethatthismaybebecausewe
refrained from aggressively filtering the pretraining data. Recall that leaving pretraining data unfiltered may
enable base models tuned to perform well on more downstream tasks (including hate speech detection),
and it carries less risk of accidentally filtering out some demographic groups. We observe that models
trained from less aggressively filtered pretraining data also required fewer examples to achieve reasonable
safety-alignment. Wereiteratethatthismotivatedchoicedoesimplythatadditionalsafetymitigationsshould
be applied before deployment of base Llama 2 models.
22

TruthfulQA ↑ToxiGen ↓
MPT7B 29.13 22.32
30B 35.25 22.61
Falcon7B 25.95 14.53
40B 40.39 23.44
Llama 17B 27.42 23.00
13B 41...<br>

In [37]:
query_engine_chunk = RetrieverQueryEngine.from_args(
    retriever_chunk, service_context=service_context
)

In [38]:
response = query_engine_chunk.query(
    "Can you tell me about the key concepts for safety finetuning"
)
print(str(response))

Retrieving with query id None: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-1
Retrieving with query id node-1: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-25
Retrieving with query id node-25: Can you tell me about the key concepts for safety finetuning


Llama.generate: prefix-match hit



Safety fine-tuning is an approach used to mitigate potential risks associated with large language models by training them to adhere to specific safety guidelines. In the context provided, the key concepts for safety fine-tuning include:

1. Supervised Safety Fine-Tuning: This technique involves gathering adversarial prompts (prompts designed to elicit undesirable behavior from the model) and safe demonstrations (prompts that elicit desirable behavior). These examples are then included in the general supervised fine-tuning process, which teaches the model to align with safety guidelines even before Reinforcement Learning with Human Feedback (RLHF) is applied. This step lays the foundation for high-quality human preference data annotation during RLHF.

By incorporating these concepts, safety fine-tuning aims to reduce the likelihood of the model generating undesirable outputs that could potentially harm users or society when the model is deployed in real-world applications.


## Eval

In [39]:
from llama_index.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
import nest_asyncio

nest_asyncio.apply()

In [42]:
eval_dataset = generate_question_context_pairs(base_nodes,llm)

100%|██████████| 93/93 [06:15<00:00,  4.03s/it]


In [43]:
eval_dataset.save_json("llama2_eval_dataset.json")

In [44]:
import pandas as pd
from llama_index.evaluation import RetrieverEvaluator, get_retrieval_results_df

# set vector retriever similarity top k to higher
top_k = 10


def display_results(names, results_arr):
    """Display results from evaluate."""

    hit_rates = []
    mrrs = []
    for name, eval_results in zip(names, results_arr):
        metric_dicts = []
        for eval_result in eval_results:
            metric_dict = eval_result.metric_vals_dict
            metric_dicts.append(metric_dict)
        results_df = pd.DataFrame(metric_dicts)

        hit_rate = results_df["hit_rate"].mean()
        mrr = results_df["mrr"].mean()
        hit_rates.append(hit_rate)
        mrrs.append(mrr)

    final_df = pd.DataFrame(
        {"retrievers": names, "hit_rate": hit_rates, "mrr": mrrs}
    )
    display(final_df)

In [45]:
# base
base_retriever = base_index.as_retriever(similarity_top_k=top_k)
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=base_retriever
)
results_base = await retriever_evaluator.aevaluate_dataset(
    eval_dataset, show_progress=True
)


100%|██████████| 212/212 [00:03<00:00, 56.64it/s]


In [46]:
# chunk
vector_retriever_chunk = vector_index_chunk.as_retriever(
    similarity_top_k=top_k
)
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever_chunk
)

results_chunk = await retriever_evaluator.aevaluate_dataset(
    eval_dataset, show_progress=True
)

  0%|          | 0/212 [00:00<?, ?it/s]

Retrieving with query id None: Question 1: What is the primary purpose of Llama 2-Chat models according to the provided context?
Retrieved node with id, entering: node-0
Retrieving with query id node-0: Question 1: What is the primary purpose of Llama 2-Chat models according to the provided context?
Retrieved node with id, entering: node-4
Retrieving with query id node-4: Question 1: What is the primary purpose of Llama 2-Chat models according to the provided context?
Retrieved node with id, entering: node-39
Retrieving with query id node-39: Question 1: What is the primary purpose of Llama 2-Chat models according to the provided context?
Retrieved node with id, entering: node-3
Retrieving with query id node-3: Question 1: What is the primary purpose of Llama 2-Chat models according to the provided context?
Retrieving with query id None: Question 2: According to the context, what category of safety violations did Llama 2-Chat show relatively more instances of, despite still maintaining

100%|██████████| 212/212 [00:18<00:00, 11.26it/s]

Retrieved node with id, entering: node-21
Retrieving with query id node-21: Question 1: In the human evaluation results shown in Figure 1, what was the 95% confidence interval and what does it indicate about the evaluation?
Retrieved node with id, entering: node-2
Retrieving with query id node-2: Question 1: In the human evaluation results shown in Figure 1, what was the 95% confidence interval and what does it indicate about the evaluation?
Retrieved node with id, entering: node-13
Retrieving with query id node-13: Question 1: In the human evaluation results shown in Figure 1, what was the 95% confidence interval and what does it indicate about the evaluation?
Retrieved node with id, entering: node-62
Retrieving with query id node-62: Question 1: In the human evaluation results shown in Figure 1, what was the 95% confidence interval and what does it indicate about the evaluation?
Retrieved node with id, entering: node-66
Retrieving with query id node-66: Question 1: In the human evalu

In [47]:
full_results_df = get_retrieval_results_df(
    [
        "Base Retriever",
        "Retriever (Chunk References)"
    ],
    [results_base, results_chunk],
)
display(full_results_df)

,retrievers,hit_rate,mrr
0,Base Retriever,0.665094,0.459406
1,Retriever (Chunk References),0.830189,0.655660


## Sentence Window Retriver

In [48]:
from llama_index.node_parser import SentenceWindowNodeParser

In [49]:
# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [50]:
node_parser

SentenceWindowNodeParser(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7d2744c0b6a0>, id_func=<function default_id_func at 0x7d285dee9c60>, sentence_splitter=<function split_by_sentence_tokenizer.<locals>.split at 0x7d27c4e43130>, window_size=3, window_metadata_key='window', original_text_metadata_key='original_text')

In [51]:
sentence_nodes = node_parser.get_nodes_from_documents(docs)

In [52]:
sentence_index = VectorStoreIndex(sentence_nodes, service_context=service_context)

In [53]:
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor

query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)


In [54]:
window_response = query_engine.query(
    "Can you tell me about the key concepts for safety finetuning"
)
print(window_response)

Llama.generate: prefix-match hit



Safety fine-tuning is an approach to ensure that an AI model aligns with safety guidelines before undergoing Reinforcement Learning from Human Feedback (RLHF). The key concepts for safety fine-tuning include:

1. Supervised Safety Fine-Tuning: This technique involves gathering adversarial prompts (prompts that may lead the model to generate unsafe responses) and safe demonstrations (prompts that lead the model to generate safe responses). These prompts are then included in the general supervised fine-tuning process (Section 3.1), which teaches the model to align with safety guidelines even before RLHF, laying the foundation for high-quality human preference data annotation.

2. Safety RLHF: After supervised safety fine-tuning, safety is integrated into the general RLHF pipeline (Section 3.2.2). This step further refines the model's understanding of safety guidelines by incorporating human feedback on both safe and unsafe responses during the RLHF process. This helps the model learn to

In [55]:
# check the original sentence that was retrieved for each node, as well as the actual window of sentences that was sent to the LLM.
window = window_response.source_nodes[0].node.metadata["window"]
sentence = window_response.source_nodes[0].node.metadata["original_text"]

print(f"Window: {window}")
print("------------------")
print(f"Original Sentence: {sentence}")

Window: AsLLMsareintegratedanddeployed,welookforwardto
continuing research that will amplify their potential for positive impact on these important social issues.
 4.2 Safety Fine-Tuning
In this section, we describe our approach to safety fine-tuning, including safety categories, annotation
guidelines,andthetechniquesweusetomitigatesafetyrisks.  Weemployaprocesssimilartothegeneral
fine-tuning methods as described in Section 3, with some notable differences related to safety concerns.
 Specifically, we use the following techniques in safety fine-tuning:
1.Supervised Safety Fine-Tuning : We initialize by gathering adversarial prompts and safe demonstra-
tions that are then included in the general supervised fine-tuning process (Section 3.1).  This teaches
themodeltoalignwithoursafetyguidelinesevenbeforeRLHF,andthuslaysthefoundationfor
high-quality human preference data annotation.
 2.Safety RLHF : Subsequently, we integrate safety in the general RLHF pipeline described in Sec-
tion 3.2.2

## API

In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://ngrok-agent.s3.amazonaws.com buster InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
40 packages can be upgraded. Run 'apt list --upgradabl

In [ ]:
!ngrok authtoken "YOUR NGROK_API"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, request, jsonify, Response,stream_with_context
from pyngrok import ngrok
from flask_cors import CORS

app = Flask(__name__)
CORS(app)
@app.route("/index")
def index():
    return "Hello"

@app.route('/generate', methods=['POST'])
def generate():
    inp = request.get_json().get("prompt")
    print(f"======Input:{inp}=======")
    index=VectorStoreIndex.from_documents(documents,service_context=service_context)
    query_engine=index.as_query_engine()
    response=query_engine.query(inp)
    return jsonify({'generated_text': str(response)})

if __name__ == '__main__':
    ngrok_tunnel = ngrok.connect(5000)
    print('Public URL:', ngrok_tunnel.public_url)
    app.run()

Public URL: https://559a-34-87-59-135.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:21:05] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:21:05] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:21:26] "OPTIONS /generate HTTP/1.1" 200 -


======Input:Hi=======


Llama.generate: prefix-match hit
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:21:30] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:21:46] "OPTIONS /generate HTTP/1.1" 200 -


======Input:who are you?=======


...

Llama.generate: prefix-match hit
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:27:14] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:27:59] "OPTIONS /generate HTTP/1.1" 200 -


======Input:what are the relevant topics in this lecture?=======


Llama.generate: prefix-match hit
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:28:04] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:28:41] "OPTIONS /generate HTTP/1.1" 200 -


======Input:can you tell the name of the project for the end semester=======


Llama.generate: prefix-match hit
INFO:werkzeug:127.0.0.1 - - [05/Feb/2024 08:28:46] "POST /generate HTTP/1.1" 200 -
